In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
import random
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [3]:
pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_14252\3002988258.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\C

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool
import torch.optim as optim
from torch_geometric.data import DataLoader

class ARGA(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim, dropout=0.2):
        super(ARGA, self).__init__()
        self.encoder = Encoder(node_features, edge_features, hidden_dim, latent_dim, dropout)
        self.decoder = Decoder(latent_dim, hidden_dim, node_features, edge_features, dropout)
        self.discriminator = Discriminator(latent_dim, hidden_dim, dropout)
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.fill_(0.01)

    def encode(self, x, edge_index, edge_attr, batch=None):
        return self.encoder(x, edge_index, edge_attr, batch)

    def decode(self, z, edge_index, batch=None):
        return self.decoder(z, edge_index, batch)

    def discriminate(self, z):
        return self.discriminator(z)

    def forward(self, x, edge_index, edge_attr, batch=None):
        z, mu = self.encode(x, edge_index, edge_attr, batch)
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index, batch)
        return reconstructed_x, reconstructed_edge_attr, z

    def generate_multiple_outputs(self, x, edge_index, edge_attr, batch=None, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, mu = self.encode(x, edge_index, edge_attr, batch)
            for _ in range(num_samples):
                # Add controlled noise to the latent space
                epsilon = torch.randn_like(z) * 0.1
                z_new = mu + epsilon

                # Decode the new latent representation
                reconstructed_x, reconstructed_edge_attr = self.decode(z_new, edge_index, batch)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class Encoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim, dropout):
        super(Encoder, self).__init__()
        
        # Edge feature processing
        self.edge_encoder = nn.Linear(edge_features, hidden_dim)
        
        # Node processing layers
        self.conv1 = SAGEConv(node_features + hidden_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        
        # Output layers
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_var = nn.Linear(hidden_dim, latent_dim)
        
        self.dropout = dropout

    def forward(self, x, edge_index, edge_attr, batch=None):
        # Process edge features
        edge_attr = F.relu(self.edge_encoder(edge_attr))
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1))).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        
        # Combine node and edge features
        x = torch.cat([x, edge_attr_expanded], dim=1)
        
        # Graph convolution layers with dropout
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        
        # Global pooling
        if batch is not None:
            x = global_mean_pool(x, batch)
        else:
            x = x.mean(dim=0, keepdim=True)
        
        # Compute mean and variance for VAE
        mu = self.fc_mu(x)
        log_var = self.fc_var(x)
        
        # Reparameterization trick
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + eps * std
        
        return z, mu

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features, dropout):
        super(Decoder, self).__init__()
        
        # Latent to hidden transformation
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Node decoder
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        
        # Edge decoder
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index, batch=None):
        # Transform latent representation
        h = self.fc(z)
        
        # Handle batching
        if batch is not None:
            h = h[batch]
        else:
            h = h.repeat(edge_index.max().item() + 1, 1)
        
        # Reconstruct node features
        reconstructed_x = self.node_decoder(h)
        
        # Reconstruct edge features
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)
        
        return reconstructed_x, reconstructed_edge_attr

class Discriminator(nn.Module):
    def __init__(self, latent_dim, hidden_dim, dropout):
        super(Discriminator, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, z):
        x = self.net(z)
        return torch.sigmoid(x)

In [5]:
import os
import json
import torch
import optuna
import multiprocessing
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.utils.data import random_split

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split

def train_arga_with_early_stopping(
    model, train_loader, val_loader, num_epochs, lr=0.001, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), 
    patience=10, beta=0.5, gamma=0.01
):
    model.to(device)
    
    gen_optimizer = optim.AdamW(
        list(model.encoder.parameters()) + list(model.decoder.parameters()),
        lr=lr, weight_decay=1e-5
    )
    disc_optimizer = optim.AdamW(model.discriminator.parameters(), lr=lr * 0.5, weight_decay=1e-5)
    
    gen_scheduler = ReduceLROnPlateau(gen_optimizer, mode="min", factor=0.5, patience=patience//2, verbose=True)
    disc_scheduler = ReduceLROnPlateau(disc_optimizer, mode="min", factor=0.5, patience=patience//2, verbose=True)

    best_val_loss = float("inf")
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        recon_loss = 0
        adv_loss = 0
        disc_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            batch_size = batch.num_graphs

            disc_optimizer.zero_grad()
            reconstructed_x, reconstructed_edge_attr, z = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            
            real_z = torch.randn_like(z)
            fake_z = z.detach()

            real_scores = model.discriminate(real_z)
            fake_scores = model.discriminate(fake_z)

            d_loss = -(torch.mean(real_scores) - torch.mean(fake_scores))
            
            alpha = torch.rand(z.size(0), 1, device=device)
            interpolated_z = alpha * real_z + (1 - alpha) * fake_z
            interpolated_z.requires_grad_(True)
            
            interpolated_scores = model.discriminate(interpolated_z)
            gradients = torch.autograd.grad(
                outputs=interpolated_scores,
                inputs=interpolated_z,
                grad_outputs=torch.ones_like(interpolated_scores),
                create_graph=True,
                retain_graph=True
            )[0]
            
            gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
            d_loss = d_loss + 10.0 * gradient_penalty

            d_loss.backward()
            disc_optimizer.step()
            disc_loss += d_loss.item()

            gen_optimizer.zero_grad()
            
            reconstructed_x, reconstructed_edge_attr, z = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            
            node_recon_loss = F.mse_loss(reconstructed_x, batch.x)
            edge_recon_loss = F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
            loss_recon = node_recon_loss + edge_recon_loss

            fake_scores = model.discriminate(z)
            loss_adv = -torch.mean(fake_scores)

            g_loss = loss_recon + gamma * loss_adv

            g_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            gen_optimizer.step()

            total_loss += g_loss.item()
            recon_loss += loss_recon.item()
            adv_loss += loss_adv.item()

        avg_total_loss = total_loss / len(train_loader)
        avg_recon_loss = recon_loss / len(train_loader)
        avg_adv_loss = adv_loss / len(train_loader)
        avg_disc_loss = disc_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, z = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                
                node_recon_loss = F.mse_loss(reconstructed_x, batch.x)
                edge_recon_loss = F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
                val_loss = node_recon_loss + edge_recon_loss
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(
            f"Epoch [{epoch+1}/{num_epochs}] - "
            f"Total Loss: {avg_total_loss:.4f}, Recon: {avg_recon_loss:.4f}, "
            f" Adv: {avg_adv_loss:.4f}, Disc: {avg_disc_loss:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}"
        )

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            print(f"No improvement. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

        gen_scheduler.step(avg_val_loss)
        disc_scheduler.step(avg_disc_loss)

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Best model state restored")

    return model


In [7]:
import optuna
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

# Function to define the objective for Optuna
def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")
    # Prepare data loaders
    pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')
    dataset = pyg_dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize the model
    model = ARGA(
        node_features=134,
        edge_features=6,
        hidden_dim=hidden_dim,
        latent_dim=latent_dim,
    )

    # Train the model with early stopping
    trained_model = train_arga_with_early_stopping(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=60,
        lr=learning_rate,
        patience=patience,
    )

    # Evaluate on the validation set
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            # batch = batch.to(trained_model.device)
            reconstructed_x, reconstructed_edge_attr, z = trained_model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )
            node_recon_loss = F.mse_loss(reconstructed_x, batch.x)
            edge_recon_loss = F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
            total_val_loss += node_recon_loss.item() + edge_recon_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Trial: {trial.number}")
    return avg_val_loss


# Main function to run the study
def run_study():
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=multiprocessing.cpu_count())  # Run with multiprocessing

    print("Best hyperparameters: ", study.best_params)
    print("Best validation loss: ", study.best_value)

    # Save the study for future use
    study.trials_dataframe().to_csv("optuna_study_results.csv")
    study.save_checkpoint("optuna_study.pkl")


if __name__ == "__main__":
    run_study()


[I 2024-12-25 11:15:28,842] A new study created in memory with name: no-name-7b9c6791-cb77-4dfc-be65-2d20e69a6992
C:\Users\sarab\AppData\Local\Temp\ipykernel_14252\3405442532.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub 


[Trial 0] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 32, 'learning_rate': 0.023763956682179514, 'batch_size': 16, 'patience': 5}
[Trial 1] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 96, 'learning_rate': 0.009958961728776203, 'batch_size': 8, 'patience': 9}


[Trial 2] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 64, 'learning_rate': 0.005503967618321748, 'batch_size': 32, 'patience': 4}

[Trial 3] Starting with parameters: {'hidden_dim': 128, 'latent_dim': 96, 'learning_rate': 0.0027882769999004573, 'batch_size': 16, 'patience': 6}

[Trial 4] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 1.5109907732722764e-05, 'batch_size': 16, 'patience': 8}

[Trial 5] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 96, 'learning_rate': 3.749678578080826e-05, 'batch_size': 16, 'patience': 9}

[Trial 6] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.0022865421710479084, 

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/60] - Total Loss: 0.2041, Recon: 0.2075,  Adv: -0.3370, Disc: 3.8552, Val Loss: 0.1360
New best model saved with validation loss: 0.1360
Epoch [1/60] - Total Loss: 0.1706, Recon: 0.1747,  Adv: -0.4145, Disc: 3.4902, Val Loss: 0.0944
New best model saved with validation loss: 0.0944
Epoch [1/60] - Total Loss: 2538.9169, Recon: 2538.9228,  Adv: -0.5890, Disc: 9.7708, Val Loss: 0.1238
New best model saved with validation loss: 0.1238
Epoch [1/60] - Total Loss: 0.2257, Recon: 0.2298,  Adv: -0.4047, Disc: 3.1070, Val Loss: 0.0984
New best model saved with validation loss: 0.0984
Epoch [1/60] - Total Loss: 0.1397, Recon: 0.1433,  Adv: -0.3633, Disc: 3.4041, Val Loss: 0.0959
New best model saved with validation loss: 0.0959
Epoch [1/60] - Total Loss: 0.1353, Recon: 0.1391,  Adv: -0.3805, Disc: 3.1490, Val Loss: 0.1001
New best model saved with validation loss: 0.1001
Epoch [2/60] - Total Loss: 0.1136, Recon: 0.1170,  Adv: -0.3391, Disc: 4.2439, Val Loss: 0.1042
New best model saved w

[I 2024-12-25 11:20:54,266] Trial 14 finished with value: 0.12391778845340014 and parameters: {'hidden_dim': 128, 'latent_dim': 128, 'learning_rate': 0.045840309081227326, 'batch_size': 32, 'patience': 4}. Best is trial 14 with value: 0.12391778845340014.


...

[I 2024-12-26 10:28:30,377] Trial 198 finished with value: 0.08865127617008045 and parameters: {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.00012635626328334176, 'batch_size': 8, 'patience': 10}. Best is trial 38 with value: 0.08475383251373257.


Trial: 198
Epoch [59/60] - Total Loss: 0.0784, Recon: 0.0883,  Adv: -0.9990, Disc: 7.2313, Val Loss: 0.0872
No improvement. Patience counter: 1/10
Epoch [60/60] - Total Loss: 0.0783, Recon: 0.0883,  Adv: -0.9990, Disc: 7.2875, Val Loss: 0.0870
No improvement. Patience counter: 2/10
Best model state restored


[I 2024-12-26 10:28:50,431] Trial 199 finished with value: 0.0870229063023414 and parameters: {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.00012253264380113202, 'batch_size': 8, 'patience': 10}. Best is trial 38 with value: 0.08475383251373257.


Trial: 199
Best hyperparameters:  {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.00018393207063419938, 'batch_size': 8, 'patience': 10}
Best validation loss:  0.08475383251373257


AttributeError: 'Study' object has no attribute 'save_checkpoint'